## 01. 트랜스포머의 등장 이유

- 트랜스포머 이전에 RNN, LSTM 네트워크가 존재했다(다음 단어 예측/next word prediction, 기계번역/machine translation, 텍스트 생성/text generation 등의 순차적 태스크에서 널리 사용되었음).
- 하지만 이러한 네트워크는 <span style='color:red'>장기 의존성 문제/long-term dependency가 있다.</span>
>**[ long-term dependency ]<br>**
RNN이 hidden state를 통해 과거의 정보를 저장할 때 문장의 길이가 길어지면 앞의 과거 정보가 마지막 시점까지 전달되지 못하는 현상.<br>
-> 모델 학습에 있어 과거의 정보를 활용하지 못하기에 긴 문장이 입력됐을 때 성능이 떨어진다.
- 순수하게 어텐션/attention만을 사용한 모델인 트랜스포머는 RNN, LSTM의 한계점을 극복하기 위해 등장했다. (원 논문: [Attention is All You Need](https://arxiv.org/abs/1706.03762))

## 02. 트랜스포머 인코더/디코더 구현해보기

In [1]:
import torch
import torch.nn as nn

/home/nstalways/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class encoder(nn.Module):
    def __init__():
        super().__init__()
    
    def forward(self, x):
    
class decoder(nn.Module):
    def __init__():
        super().__init__()
        
    def forward(self, x):

In [ ]:
class Transformer(nn.Module):
    def __init__():
        super().__init__()
        self.encoder = 
        self.decoder = 
        
    def forward(self, x):
        